#Hackathon : Evènements marquants aux JO (Questions)


###Importation des packages et modules

In [1]:
!pip install pyspark



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.2 MB/s eta 0:00:00

  Preparing metadata (setup.py) ... done




  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=6280c1ee95e7bf38589fea1b3653c2587bce8dab9618a66ba4893a0b617c6734

  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6

Successfully built pyspark




In [2]:
import pandas as pd
import re
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd


from pyspark.sql import SparkSession
from pyspark.sql.functions import col, isnan, when, regexp_replace, count, avg, mode
from pyspark.sql.types import IntegerType

from bs4 import BeautifulSoup
from io import StringIO
from lxml import etree


from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, BooleanType

from pyspark.sql.functions import col, count, when, length, trim, isnan, regexp_extract, udf, coalesce, lit, sum, avg, expr, upper, trim, sum as spark_sum
 

from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

from matplotlib.colors import ListedColormap

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


###Importation des données

In [282]:
# Initialiser une session Spark
spark = SparkSession.builder.appName("KaggleSparkXML").getOrCreate()
 
# Définir le chemin du fichier XML
file_path = '/kaggle/input/jeux-ol/olympic_hosts.xml'
 
# Lire le fichier XML
tree = etree.parse(file_path)
root = tree.getroot()
 
# Convertir les données XML en une liste de dictionnaires
data = [{child.tag: child.text for child in root[i]} for i in range(len(root))]
 
# Créer un DataFrame Spark à partir des données
host = spark.createDataFrame(data)
 
# Afficher les données
host.show()

24/05/30 14:27:59 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


+--------------------+------------------+-------------------+-----------+-------------------+--------------------+---------+-----+
|       game_end_date|     game_location|          game_name|game_season|          game_slug|     game_start_date|game_year|index|
+--------------------+------------------+-------------------+-----------+-------------------+--------------------+---------+-----+
|2022-02-20T12:00:00Z|             China|       Beijing 2022|     Winter|       beijing-2022|2022-02-04T15:00:00Z|     2022|    0|
|2021-08-08T14:00:00Z|             Japan|         Tokyo 2020|     Summer|         tokyo-2020|2021-07-23T11:00:00Z|     2020|    1|
|2018-02-25T08:00:00Z| Republic of Korea|   PyeongChang 2018|     Winter|   pyeongchang-2018|2018-02-08T23:00:00Z|     2018|    2|
|2016-08-21T21:00:00Z|            Brazil|           Rio 2016|     Summer|           rio-2016|2016-08-05T12:00:00Z|     2016|    3|
|2014-02-23T16:00:00Z|Russian Federation|         Sochi 2014|     Winter|         s

In [283]:

file_path_html = '/kaggle/input/jeux-ol/olympic_results.html'

# Lire le html
with open(file_path_html, 'r') as file:
    html_content = file.read()

# Parse avec BeautifulSoup
soup = BeautifulSoup(html_content, 'lxml')

# Extraire le tableau
table = soup.find('table')

# Lire le tableau dans une df pandas
df_pandas = pd.read_html(StringIO(str(table)))[0]

# Check data types
print(df_pandas.dtypes)

Unnamed: 0                int64
discipline_title         object
event_title              object
slug_game                object
participant_type         object
medal_type               object
athletes                 object
rank_equal               object
rank_position            object
country_name             object
country_code             object
country_3_letter_code    object
athlete_url              object
athlete_full_name        object
value_unit               object
value_type               object
dtype: object


In [ ]:
# Convertir les cols 
df_pandas['Unnamed: 0'] = pd.to_numeric(df_pandas['Unnamed: 0'], errors='coerce').fillna(0).astype(int)
df_pandas['rank_position'] = pd.to_numeric(df_pandas['rank_position'], errors='coerce').fillna(0).astype(int)
df_pandas['rank_equal'] = df_pandas['rank_equal'].astype(str).apply(lambda x: x.lower() in ['true', '1', 'yes'])

# Definir la schema
schema = StructType([
    StructField("Unnamed: 0", IntegerType(), True),
    StructField("discipline_title", StringType(), True),
    StructField("event_title", StringType(), True),
    StructField("slug_game", StringType(), True),
    StructField("participant_type", StringType(), True),
    StructField("medal_type", StringType(), True),
    StructField("athletes", StringType(), True),
    StructField("rank_equal", BooleanType(), True),
    StructField("rank_position", IntegerType(), True),
    StructField("country_name", StringType(), True),
    StructField("country_code", StringType(), True),
    StructField("country_3_letter_code", StringType(), True),
    StructField("athlete_url", StringType(), True),
    StructField("athlete_full_name", StringType(), True),
    StructField("value_unit", StringType(), True),
    StructField("value_type", StringType(), True)
])

# Convert the Pandas DataFrame to a Spark DataFrame using the defined schema
resultats = spark.createDataFrame(df_pandas, schema=schema)

resultats.show()

In [4]:
# Initialiser une session Spark
spark = SparkSession.builder \
    .appName("Check Missing Data") \
    .getOrCreate()

# Chemins vers les fichiers
olympics = "/content/drive/MyDrive/olympics.json"
medals = "/content/drive/MyDrive/medals.json"
summer = "/content/drive/MyDrive/summer_results.json"
winter = "/content/drive/MyDrive/winter_results.json"
athlete = "/content/drive/MyDrive/olympic_athletes_clean.json"

# Charger les données JSON dans des DataFrames
df_olympics = spark.read.json(olympics)
df_medals = spark.read.json(medals)
df_summer = spark.read.json(summer)
df_winter = spark.read.json(winter)
df_athlete = spark.read.json(athlete)

# Afficher les DataFrames
#df_olympics.show(truncate=False)
df_medals.show(truncate=False)
#df_summer.show(truncate=False)
#df_winter.show(truncate=False)
df_athlete.show(truncate=False)

+----------+--------------------+-----------------------------------------------------+---------------------+------------+------------------------+----------------+------------+----------------------+----------+-----------------+----------------+------------+

|Unnamed: 0|athlete_full_name   |athlete_url                                          |country_3_letter_code|country_code|country_name            |discipline_title|event_gender|event_title           |medal_type|participant_title|participant_type|slug_game   |

+----------+--------------------+-----------------------------------------------------+---------------------+------------+------------------------+----------------+------------+----------------------+----------+-----------------+----------------+------------+

|0         |Stefania CONSTANTINI|https://olympics.com/en/athletes/stefania-constantini|ITA                  |IT          |Italy                   |Curling         |Mixed       |Mixed Doubles         |GOLD      |Italy 

###1. La France a organisé 6 JO : 3 d’hiver et 3 d’été (en comptant celui de 2024) ?

In [21]:
# Sélectionner les occurrences uniques des jeux
unique_games = df_olympics.filter(df_olympics["game_location"] == "France").select("game_slug", "game_location", "game_season").distinct()

# Regrouper par emplacement du jeu et saison pour obtenir le décompte des jeux uniques organisés
games_by_country_season = unique_games.groupBy("game_location", "game_season").count().withColumnRenamed("count", "total_jeux")

# Afficher les résultats
games_by_country_season.show()
print("\nLa France a bien organisé 6 JO : 3 d'hiver + 2 d'été + 1 d'été avec 2024")

+-------------+-----------+----------+

|game_location|game_season|total_jeux|

+-------------+-----------+----------+

|       France|     Winter|         3|

|       France|     Summer|         2|

+-------------+-----------+----------+





La France a bien organisé 6 JO : 3 d'hiver + 2 d'été + 1 d'été avec 2024


###2. La France est le 2è pays qui a organisé le plus de JO après les USA (8 JO) ?

In [22]:
# Compter le nombre de JO par pays
jo_counts = df_olympics.groupBy("game_location").count()

# Afficher les résultats
jo_counts.orderBy(jo_counts["count"].desc()).show()
print("\nLa France est le deuxième pays qui organise le plus de JO")

+------------------+-----+

|     game_location|count|

+------------------+-----+

|              NULL|    8|

|     United States|    8|

|            France|    5|

|             Japan|    4|

|     Great Britain|    3|

|             Italy|    3|

|            Canada|    3|

| Republic of Korea|    2|

|           Germany|    2|

|            Greece|    2|

|             China|    2|

|            Norway|    2|

|       Switzerland|    2|

|           Austria|    2|

|            Sweden|    1|

|           Belgium|    1|

|     game_location|    1|

|           Finland|    1|

|             Spain|    1|

|Russian Federation|    1|

+------------------+-----+

only showing top 20 rows





La France est le deuxième pays qui organise le plus de JO


###3. Les JO d’hiver sont nés à Chamonix en 1924 ?

In [24]:
# Filter the data for the 1924 Winter Olympics
chamonix_1924_data = df_olympics.filter((col('game_year') == 1924) & (col('game_season') == 'Winter'))

# Check if the 1924 Winter Olympics were held in Chamonix, France
is_chamonix_1924 = chamonix_1924_data.filter((col('game_location') == 'France') & (col('game_slug') == 'chamonix-1924')).count() > 0

print(f"Les JO d’hiver sont nés à Chamonix en 1924 : {is_chamonix_1924}")

Les JO d’hiver sont nés à Chamonix en 1924 : True


###4. JO de Paris, en 1900 : les femmes peuvent participer aux JO ?

In [7]:
# Filter the data for Paris 1900 Olympics
paris_1900_df = df_medals.filter(df_medals['slug_game'].contains('paris-1900'))
paris_1900_df.show(5)

# Check for the presence of women in the event_gender column
women_participation = paris_1900_df.filter(paris_1900_df['event_gender'].contains('Women')).count()

# result
if women_participation > 0:
    print("Les femmes ont participé aux Jeux Olympiques de Paris en 1900.")
else:
    print("Les femmes n'ont pas participé aux Jeux Olympiques de Paris en 1900.")

+----------+--------------------+--------------------+---------------------+------------+------------+----------------+------------+--------------------+----------+-----------------+----------------+----------+

|Unnamed: 0|   athlete_full_name|         athlete_url|country_3_letter_code|country_code|country_name|discipline_title|event_gender|         event_title|medal_type|participant_title|participant_type| slug_game|

+----------+--------------------+--------------------+---------------------+------------+------------+----------------+------------+--------------------+----------+-----------------+----------------+----------+

|     21279|     Lucien MERIGNAC|https://olympics....|                  FRA|          FR|      France|         Fencing|         Men|    foil masters men|      GOLD|             NULL|         Athlete|paris-1900|

|     21280|Alphonse KIRCHHOFFER|https://olympics....|                  FRA|          FR|      France|         Fencing|         Men|    foil masters men

###5. Seuls 4 athlètes ont remporté des médailles à la fois aux JO d’hiver et d’été. Une seule d’entre eux, Christa Ludinger-Rothenburger, a remporté des médailles au cours de la même année ?

In [284]:
# Perform the join on the 'slug_game' column
joined_df = resultats.join(host, resultats.slug_game == host.game_slug)
# Filter only rows with valid medals (ignoring 0)
medal_winners = joined_df.filter(joined_df.medal_type.isin(["GOLD", "SILVER", "BRONZE"]))

# Show a sample of the medal winners
medal_winners.show(10, truncate=False)

24/05/30 14:32:42 WARN TaskSetManager: Stage 3773 contains a task of very large size (4943 KiB). The maximum recommended task size is 1000 KiB.


+----------+----------------+------------------+----------+----------------+----------+----------+-------------+------------------------+---------------------+--------------------------------------------------+-----------------+--------------------+------------------+----------+-----------+----------+--------------------+---------+-----+
|Unnamed: 0|discipline_title|event_title       |slug_game |participant_type|medal_type|rank_equal|rank_position|country_name            |country_3_letter_code|athlete_url                                       |athlete_full_name|game_end_date       |game_location     |game_name |game_season|game_slug |game_start_date     |game_year|index|
+----------+----------------+------------------+----------+----------------+----------+----------+-------------+------------------------+---------------------+--------------------------------------------------+-----------------+--------------------+------------------+----------+-----------+----------+------------------

In [287]:
# Group by athlete and game_season to find athletes who have won medals in both seasons
athlete_season_medals = medal_winners.select("athlete_full_name", "game_season").distinct()

# Count the number of distinct seasons each athlete has won medals in
athlete_medal_count = athlete_season_medals.groupBy("athlete_full_name").count()

# Filter for athletes who have won medals in both Winter and Summer Olympics
athletes_both_seasons = athlete_medal_count.filter("count > 1")
athletes_both_seasons.show(truncate=False)

# Check if Christa Ludinger-Rothenburger won medals in both Winter and Summer Olympics in the same year
christa_medals = medal_winners.filter(medal_winners.athlete_full_name == "Christa Ludinger-Rothenburger")
christa_medals_in_same_year = christa_medals.groupBy("game_year", "game_season").count().groupBy("game_year").agg({"game_season": "count"}).filter("count(game_season) > 1")
christa_medals_in_same_year.show(truncate=False)


24/05/30 14:32:54 WARN TaskSetManager: Stage 3809 contains a task of very large size (4943 KiB). The maximum recommended task size is 1000 KiB.


+---------------------------+-----+
|athlete_full_name          |count|
+---------------------------+-----+
|Gillis GRAFSTRÖM           |2    |
|Christa LUDING-ROTHENBURGER|2    |
|Gunnar LARSSON             |2    |
|Clara HUGHES               |2    |
|NaN                        |2    |
+---------------------------+-----+



24/05/30 14:32:56 WARN TaskSetManager: Stage 3822 contains a task of very large size (4943 KiB). The maximum recommended task size is 1000 KiB.


+---------+------------------+
|game_year|count(game_season)|
+---------+------------------+
+---------+------------------+



In [288]:
# Filter records for Christa Luding-Rothenburger
christa_medals = medal_winners.filter(medal_winners.athlete_full_name == "Christa LUDING-ROTHENBURGER")

# Show the detailed records for Christa Luding-Rothenburger
christa_medals.show(truncate=False)


24/05/30 14:33:02 WARN TaskSetManager: Stage 3824 contains a task of very large size (4943 KiB). The maximum recommended task size is 1000 KiB.


+----------+----------------+------------+----------------+----------------+----------+----------+-------------+------------------------------------+---------------------+-----------------------------------------------+---------------------------+--------------------+-----------------+----------------+-----------+----------------+--------------------+---------+-----+
|Unnamed: 0|discipline_title|event_title |slug_game       |participant_type|medal_type|rank_equal|rank_position|country_name                        |country_3_letter_code|athlete_url                                    |athlete_full_name          |game_end_date       |game_location    |game_name       |game_season|game_slug       |game_start_date     |game_year|index|
+----------+----------------+------------+----------------+----------------+----------+----------+-------------+------------------------------------+---------------------+-----------------------------------------------+---------------------------+-------------

In [289]:
# Group by game_year and game_season for Christa Luding-Rothenburger
christa_medals_by_year_season = christa_medals.groupBy("game_year", "game_season").count()

# Show the detailed medal records grouped by year and season
christa_medals_by_year_season.show(truncate=False)

# Check if she won medals in both Winter and Summer Olympics in the same year
christa_medals_in_same_year = christa_medals_by_year_season.groupBy("game_year").agg({"game_season": "count"}).filter("count(game_season) > 1")

# Show the results
christa_medals_in_same_year.show(truncate=False)

24/05/30 14:33:18 WARN TaskSetManager: Stage 3830 contains a task of very large size (4943 KiB). The maximum recommended task size is 1000 KiB.


+---------+-----------+-----+
|game_year|game_season|count|
+---------+-----------+-----+
|1992     |Winter     |1    |
|1984     |Winter     |1    |
|1988     |Winter     |2    |
|1988     |Summer     |1    |
+---------+-----------+-----+



24/05/30 14:33:20 WARN TaskSetManager: Stage 3839 contains a task of very large size (4943 KiB). The maximum recommended task size is 1000 KiB.


+---------+------------------+
|game_year|count(game_season)|
+---------+------------------+
|1988     |2                 |
+---------+------------------+



 ###  Réponse Q5 : Oui, 4 athlètes ont remporté des médailles aux Jeux Olympiques au cours des deux saisons. Oui |Christa LUDING-ROTHENBURGER a gagné à la fois en été et en hiver en 1988

###6. De 1924 à 1992, les JO d’hiver et d’été avaient lieu au cours de la même année. Désormais, ils sont organisés selon des cycles distincts avec une alternance de 2 ans ?

In [8]:
# Filtrer les données des Jeux olympiques jusqu'en 1992
olympics_until_1992 = df_olympics.filter(df_olympics["game_year"] <= 1992)

# Compter les occurrences par année et saison
year_season_counts = olympics_until_1992.groupBy("game_year", "game_season").count()
print("Occurrences par année et saison jusqu'en 1992 :")

year_season_counts.orderBy("game_year").show()

# Filtrer les données des Jeux olympiques après 1992
olympics_after_1992 = df_olympics.filter(df_olympics["game_year"] > 1992)

# Vérifier si les Jeux olympiques d'été et d'hiver se déroulent selon des cycles distincts avec une alternance de 2 ans
summer_olympics_years = olympics_after_1992.filter(olympics_after_1992["game_season"] == "Summer").select("game_year").distinct().orderBy("game_year").collect()
winter_olympics_years = olympics_after_1992.filter(olympics_after_1992["game_season"] == "Winter").select("game_year").distinct().orderBy("game_year").collect()

# Convertir les années en entiers
summer_years = [int(row["game_year"]) for row in summer_olympics_years]
winter_years = [int(row["game_year"]) for row in winter_olympics_years]

# Créer un DataFrame avec les années des Jeux olympiques d'été et d'hiver
from pyspark.sql import Row

# Créer des lignes pour les Jeux olympiques d'été et hiver
summer_rows = [Row(season="Summer", year=year) for year in summer_years]
winter_rows = [Row(season="Winter", year=year) for year in winter_years]

# Combiner les lignes
all_rows = summer_rows + winter_rows

# Créer un DataFrame à partir des lignes combinées
result_df = spark.createDataFrame(all_rows)

# Afficher le DataFrame
print("\nTableau des dates des Jeux olympiques après 1992 :")
result_df.show()

Occurrences par année et saison jusqu'en 1992 :

+---------+-----------+-----+

|game_year|game_season|count|

+---------+-----------+-----+

|     1896|     Summer|    1|

|     1900|     Summer|    1|

|     1904|     Summer|    1|

|     1908|     Summer|    1|

|     1912|     Summer|    1|

|     1920|     Summer|    1|

|     1924|     Winter|    1|

|     1924|     Summer|    1|

|     1928|     Summer|    1|

|     1928|     Winter|    1|

|     1932|     Winter|    1|

|     1932|     Summer|    1|

|     1936|     Summer|    1|

|     1936|     Winter|    1|

|     1948|     Summer|    1|

|     1948|     Winter|    1|

|     1952|     Summer|    1|

|     1952|     Winter|    1|

|     1956|     Winter|    1|

|     1956|     Summer|    1|

+---------+-----------+-----+

only showing top 20 rows





Tableau des dates des Jeux olympiques après 1992 :

+------+----+

|season|year|

+------+----+

|Summer|1996|

|Summer|2000|

|Summer|2004|

|Summer|2008|

|Summer|2012|

|Summ

###7. Tarzan lui-même a participé aux JO. En effet, Johnny Weissmuller, ancien athlète devenu acteur et ayant interprété 12 films de Tarzan, a remporté 5 médailles d’or en natation dans les années 1920 ?

In [9]:
# Filtrer les résultats pour Johnny Weissmuller
johnny_results = df_medals.filter(df_medals["athlete_url"].contains("johnny-weissmuller"))

# Filtrer pour les médailles d'or en natation
johnny_gold_medals = johnny_results.filter((johnny_results["medal_type"] == "GOLD") & (johnny_results["discipline_title"] == "Swimming"))

# Compter les médailles d'or
gold_medal_count = johnny_gold_medals.count()

# Afficher les résultats
print(f"Johnny Weissmuller a remporté {gold_medal_count} médailles d'or en natation.")

Johnny Weissmuller a remporté 3 médailles d'or en natation.


###8. Les JO de Londres de 2012 ont été les 1ers JO durant lesquels tous les pays participants ont envoyé des athlètes de sexe féminin ?

In [16]:
# Filter the data for the 2012 London Olympics
year_2012_data = df_medals.filter(df_medals['slug_game'] == 'london-2012')

# Get the unique country codes for all participants in 2012
countries_2012 = year_2012_data.select('country_3_letter_code').distinct().rdd.flatMap(lambda x: x).collect()

# Get the unique country codes for female athletes in 2012
countries_with_female_athletes = year_2012_data.filter(df_medals['participant_type'] == 'Athlete')\
                                               .select('country_3_letter_code').distinct().rdd.flatMap(lambda x: x).collect()

# Check if all countries sent female athletes
all_female_athletes = set(countries_2012) == set(countries_with_female_athletes)

print(f"Tous les pays participants en 2012 ont envoyé des athlètes de sexe féminin : {all_female_athletes}")

Tous les pays participants en 2012 ont envoyé des athlètes de sexe féminin : False


###9. Les sports suivants ne font (malheureusement) plus partie des J.O : la natation synchronisée en solo, le tir à la corde, la corde à grimper, la montgolfière, le duel au pistolet, le vélo tandem, la course d’obstacles à la nage et le plongeon à distance. Par chance, le tir au pigeon n’a été mis en place qu’une seule fois pendant les Jeux Olympiques de Paris de 1900 ?

In [9]:
paris_1900_df = df_medals.filter(df_medals['slug_game'].contains('paris-1900'))
paris_1900_df.show(5)

# List of sports to check
sports_to_check = [
    "natation synchronisée en solo", "tir à la corde", "corde à grimper",
    "montgolfière", "duel au pistolet", "vélo tandem",
    "course d'obstacles à la nage", "plongeon à distance", "tir au pigeon"
]

# Check for the presence of these sports in the event_title column
for sport in sports_to_check:
    sport_presence = paris_1900_df.filter(paris_1900_df['event_title'].contains(sport)).count()
    if sport_presence > 0:
        print(f"Le sport '{sport}' a été disputé aux Jeux Olympiques de Paris en 1900.")
    else:
        print(f"Le sport '{sport}' n'a pas été disputé aux Jeux Olympiques de Paris en 1900.")

+----------+--------------------+--------------------+---------------------+------------+------------+----------------+------------+--------------------+----------+-----------------+----------------+----------+

|Unnamed: 0|   athlete_full_name|         athlete_url|country_3_letter_code|country_code|country_name|discipline_title|event_gender|         event_title|medal_type|participant_title|participant_type| slug_game|

+----------+--------------------+--------------------+---------------------+------------+------------+----------------+------------+--------------------+----------+-----------------+----------------+----------+

|     21279|     Lucien MERIGNAC|https://olympics....|                  FRA|          FR|      France|         Fencing|         Men|    foil masters men|      GOLD|             NULL|         Athlete|paris-1900|

|     21280|Alphonse KIRCHHOFFER|https://olympics....|                  FRA|          FR|      France|         Fencing|         Men|    foil masters men

###10. Les Jeux de 2016, à Rio, marqueront la 1è manifestation des JO en Amérique du Sud ?

In [21]:
# Filter the data for the 2016 Rio Olympics
rio_2016_data = df_olympics.filter(col('game_slug') == 'rio-2016')

# Check if 'Brazil' hosted the 2016 Olympics
hosted_by_brazil = rio_2016_data.filter(col('game_location') == 'Brazil').count() > 0

# Check if there were any previous Olympics hosted by Brazil
previous_olympics_in_brazil = df_olympics.filter((col('game_location') == 'Brazil') & (col('game_slug') != 'rio-2016')).count() == 0

# Determine if the 2016 Rio Olympics were the first in South America
is_first_south_america = hosted_by_brazil and previous_olympics_in_brazil

print(f"Les Jeux de 2016 à Rio marqueront la 1ère manifestation des JO en Amérique du Sud : {is_first_south_america}")

Les Jeux de 2016 à Rio marqueront la 1ère manifestation des JO en Amérique du Sud : True


###11. Pendant les 17 jours des JO d’été de 2016, 10 500 athlètes, originaires de 205 pays, représenteront 42 sports différents et participeront à 306 épreuves, à Rio ?

In [299]:
# Filter the dataset for the 2016 Summer Olympics (Rio 2016)
rio_2016 = resultats.filter(resultats.slug_game == "rio-2016")

# Total number of distinct athletes
total_athletes = rio_2016.select("athlete_full_name").distinct().count()

# Number of participating countries using 'country_3_letter_code'
total_countries = rio_2016.select("country_3_letter_code").distinct().count()

# Number of different sports
total_sports = rio_2016.select("discipline_title").distinct().count()

# Number of events
total_events = rio_2016.select("event_title").distinct().count()

(total_athletes, total_countries, total_sports, total_events)

24/05/30 14:42:38 WARN TaskSetManager: Stage 3919 contains a task of very large size (4943 KiB). The maximum recommended task size is 1000 KiB.
24/05/30 14:42:39 WARN TaskSetManager: Stage 3925 contains a task of very large size (4943 KiB). The maximum recommended task size is 1000 KiB.
24/05/30 14:42:40 WARN TaskSetManager: Stage 3931 contains a task of very large size (4943 KiB). The maximum recommended task size is 1000 KiB.
24/05/30 14:42:41 WARN TaskSetManager: Stage 3937 contains a task of very large size (4943 KiB). The maximum recommended task size is 1000 KiB.


(5908, 199, 41, 289)

### Réponse Q 11 : non , pendant les JO d'été 2016 5908 athletes, originaires de 199 pays, répresentants 41 sports ont participé a 289 épreuves.

#Questions ouvertes

###1. Combien de médailles la France a remporté : en tout, en Or, en argent et en Bronze (depuis le début des JO) ?

In [10]:
# Filtrer les médailles pour la France
french_medals = df_medals.filter(df_medals["country_3_letter_code"] == "FRA")

# Compter les médailles totales
total_medals = french_medals.count()

# Compter les médailles par type
medals_by_type = french_medals.groupBy("medal_type").count()

# Afficher les résultats
print(f"La France a remporté {total_medals} médailles en tout.")
medals_by_type.show()

La France a remporté 952 médailles en tout.

+----------+-----+

|medal_type|count|

+----------+-----+

|    SILVER|  305|

|      GOLD|  282|

|    BRONZE|  365|

+----------+-----+




###2. Lors de quelle JO la France a eu le plus (le moins) de succès ? et 3. Peut-on considérer que la France est la grande spécialiste de certaine(s) discipline(s) sportive(s), comparativement aux autres notions ? Laquelle / Lesquelles ?

In [10]:
france_df = df_medals.filter(df_medals['country_3_letter_code'] == 'FRA')

# Filter the data for other countries
other_countries_df = df_medals.filter(df_medals['country_3_letter_code'] != 'FRA')

# Group by discipline_title and count the number of medals for France
france_medals_by_discipline = france_df.groupBy('discipline_title').count().withColumnRenamed('count', 'france_medal_count')

# Group by discipline_title and count the number of medals for other countries
other_medals_by_discipline = other_countries_df.groupBy('discipline_title').count().withColumnRenamed('count', 'other_medal_count')

# Join the two DataFrames on discipline_title
comparison_df = france_medals_by_discipline.join(other_medals_by_discipline, 'discipline_title', 'outer').fillna(0)

# Calculate the total medal count and the proportion of medals won by France
comparison_df = comparison_df.withColumn('total_medal_count', col('france_medal_count') + col('other_medal_count'))
comparison_df = comparison_df.withColumn('france_medal_proportion', col('france_medal_count') / col('total_medal_count'))

# Show the disciplines where France has a high proportion of medals
comparison_df.orderBy('france_medal_proportion', ascending=False).show()

+--------------------+------------------+-----------------+-----------------+-----------------------+

|    discipline_title|france_medal_count|other_medal_count|total_medal_count|france_medal_proportion|

+--------------------+------------------+-----------------+-----------------+-----------------------+

|             Croquet|                 8|                0|                8|                    1.0|

|     Military Patrol|                 1|                2|                3|     0.3333333333333333|

|Equestrian  Vaulting|                 2|                4|                6|     0.3333333333333333|

|   Water Motorsports|                 1|                2|                3|     0.3333333333333333|

|             Fencing|               123|              545|              668|    0.18413173652694612|

|        Canoe Slalom|                22|              110|              132|    0.16666666666666666|

|        Rugby Sevens|                 1|                5|              

###4. Quelles sont les sports les plus dominants dans les JO au fil des ans ?

In [41]:
# Grouper par discipline_title et compter le nombre de médailles
sports_medal_counts = df_medals.groupBy('discipline_title').agg(count('*').alias('total_medal_count'))

# Trier les sports par nombre total de médailles en ordre décroissant
sorted_sports_medal_counts = sports_medal_counts.orderBy(col('total_medal_count').desc())

# Afficher les sports les plus dominants
sorted_sports_medal_counts.show(10)

+-------------------+-----------------+

|   discipline_title|total_medal_count|

+-------------------+-----------------+

|          Athletics|             3080|

|           Swimming|             1763|

|          Wrestling|             1356|

|             Rowing|             1072|

|             Boxing|              996|

|Gymnastics Artistic|              979|

|           Shooting|              871|

|       Canoe Sprint|              799|

|            Sailing|              771|

|      Weightlifting|              679|

+-------------------+-----------------+

only showing top 10 rows




# Question bonus

In [291]:
# Define the URL of the page to scrape
url = "https://statisticstimes.com/economy/gdp-indicators-2020.php"

# Send a request to fetch the HTML content
response = requests.get(url)
response.raise_for_status()  # Check that the request was successful

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

# Find the table in the HTML
table = soup.find('table', {'id': 'table_id'})

# Extract headers
header_rows = table.find_all('tr')

# First header row
first_header = [th.get_text(strip=True) for th in header_rows[0].find_all('th')]

# Second header row (subheaders)
second_header = [th.get_text(strip=True) for th in header_rows[1].find_all('th')]

# Print headers for debugging
print("First header:", first_header)
print("Second header:", second_header)


multi_level_headers = []

# Manually define the headers based on the order shown in the image
multi_level_headers = [
    ('Country/Economy', ''),
    ('GDP 2020 (billions of $/Int. $)', 'Nominal'),
    ('GDP 2020 (billions of $/Int. $)', 'Rank'),
    ('GDP 2020 (billions of $/Int. $)', 'PPP'),
    ('GDP 2020 (billions of $/Int. $)', 'Rank'),
    ('GDP per capita 2020 ($/Int. $)', 'Nominal'),
    ('GDP per capita 2020 ($/Int. $)', 'Rank'),
    ('GDP per capita 2020 ($/Int. $)', 'PPP'),
    ('GDP per capita 2020 ($/Int. $)', 'Rank'),
    ('GDP Growth (%)', '2020'),
    ('GDP Growth (%)', 'Rank')
]

# Print the multi-level headers for debugging
print("Multi-level headers:", multi_level_headers)

# Extract table rows
rows = []
for row in table.find('tbody').find_all('tr'):
    cells = row.find_all('td')
    row_data = [cell.get_text(strip=True) for cell in cells]
    rows.append(row_data)

# Print the first few rows for debugging
print("First few rows:", rows[:5])

# Create a pandas DataFrame from the extracted data
df = pd.DataFrame(rows, columns=pd.MultiIndex.from_tuples(multi_level_headers))

# Display the DataFrame
print(df.head())

First header: ['Country/Economy', 'GDP 2020 (billions of $/Int. $)', 'GDP per capita 2020 ($/Int. $)', 'GDP Growth (%)']
Second header: ['Nominal', 'Rank', 'PPP', 'Rank', 'Nominal', 'Rank', 'PPP', 'Rank', '2020', 'Rank']
Multi-level headers: [('Country/Economy', ''), ('GDP 2020 (billions of $/Int. $)', 'Nominal'), ('GDP 2020 (billions of $/Int. $)', 'Rank'), ('GDP 2020 (billions of $/Int. $)', 'PPP'), ('GDP 2020 (billions of $/Int. $)', 'Rank'), ('GDP per capita 2020 ($/Int. $)', 'Nominal'), ('GDP per capita 2020 ($/Int. $)', 'Rank'), ('GDP per capita 2020 ($/Int. $)', 'PPP'), ('GDP per capita 2020 ($/Int. $)', 'Rank'), ('GDP Growth (%)', '2020'), ('GDP Growth (%)', 'Rank')]
First few rows: [['Afghanistan', '19.132', '114', '78.729', '98', '581', '184', '2,390', '174', '-5.000', '109'], ['Albania', '15.147', '126', '40.737', '117', '5,287', '95', '14,218', '90', '-3.500', '83'], ['Algeria', '144.294', '58', '491.459', '43', '3,263', '129', '11,112', '109', '-5.992', '124'], ['Angola', 

In [293]:
selected_columns = [
    ('Country/Economy', ''),
    ('GDP 2020 (billions of $/Int. $)', 'Nominal'),
    ('GDP 2020 (billions of $/Int. $)', 'PPP'),
    ('GDP per capita 2020 ($/Int. $)', 'Nominal'),
    ('GDP per capita 2020 ($/Int. $)', 'PPP'),
    ('GDP Growth (%)', '2020')
]

# Select the relevant columns
df_selected = df[selected_columns]

# Flatten the multi-level columns
df_selected.columns = ['country', 'GDP_Nominal', 'GDP_PPP', 'GDP_per_capita_Nominal', 'GDP_per_capita_PPP', 'GDP_Growth_2020']



In [294]:
# Convert the pandas DataFrame 'df_selected' to a PySpark DataFrame
gdp_spark_df = spark.createDataFrame(df_selected)

# Convert columns to numeric types
from pyspark.sql.functions import col

gdp_spark_df = gdp_spark_df.withColumn("GDP_Nominal", col("GDP_Nominal").cast("float")) \
                           .withColumn("GDP_PPP", col("GDP_PPP").cast("float")) \
                           .withColumn("GDP_per_capita_Nominal", col("GDP_per_capita_Nominal").cast("float")) \
                           .withColumn("GDP_per_capita_PPP", col("GDP_per_capita_PPP").cast("float")) \
                           .withColumn("GDP_Growth_2020", col("GDP_Growth_2020").cast("float"))

# Display the prepared GDP DataFrame
gdp_spark_df.show()


+-------------------+-----------+-------+----------------------+------------------+---------------+
|            country|GDP_Nominal|GDP_PPP|GDP_per_capita_Nominal|GDP_per_capita_PPP|GDP_Growth_2020|
+-------------------+-----------+-------+----------------------+------------------+---------------+
|        Afghanistan|     19.132| 78.729|                 581.0|              NULL|           -5.0|
|            Albania|     15.147| 40.737|                  NULL|              NULL|           -3.5|
|            Algeria|    144.294|491.459|                  NULL|              NULL|         -5.992|
|             Angola|      62.44|215.105|                  NULL|              NULL|         -3.975|
|Antigua and Barbuda|       1.39|  1.826|                  NULL|              NULL|        -17.273|
|          Argentina|    388.279|941.848|                  NULL|              NULL|         -9.956|
|            Armenia|     12.339|  39.38|                  NULL|              NULL|         -7.571|


In [295]:
# Replace NaN, empty string, and null values in 'medal_type' column with 0
resultats = resultats.withColumn("medal_type", when(col("medal_type").isNull(), 0)
                                              .when(col("medal_type") == "", 0)
                                              .otherwise(col("medal_type")))

# Verify the replacements by showing distinct values in the 'medal_type' column
resultats.select("medal_type").distinct().show()

24/05/30 14:38:18 WARN TaskSetManager: Stage 3877 contains a task of very large size (4943 KiB). The maximum recommended task size is 1000 KiB.


+----------+
|medal_type|
+----------+
|         0|
|    SILVER|
|      GOLD|
|    BRONZE|
+----------+



In [296]:
# Replace NaN, empty string, and null values in 'medal_type' column with 0
resultats = resultats.withColumn("medal_type", when(col("medal_type").isNull(), 0)
                                              .when(isnan(col("medal_type")), 0)
                                              .when(col("medal_type") == "", 0)
                                              .otherwise(col("medal_type")))

# Verify the replacements by showing distinct values in the 'medal_type' column
resultats.select("medal_type").distinct().show()



24/05/30 14:38:32 WARN TaskSetManager: Stage 3880 contains a task of very large size (4943 KiB). The maximum recommended task size is 1000 KiB.


+----------+
|medal_type|
+----------+
|         0|
|    SILVER|
|      GOLD|
|    BRONZE|
+----------+



In [297]:
# Show schemas for both DataFrames
gdp_spark_df.printSchema()
resultats.printSchema()

# Convert the country names and country codes to uppercase for consistency
gdp_spark_df = gdp_spark_df.withColumn('country', upper(col('country')))
resultatsf = resultats.withColumn('country_name', upper(col('country_name')))

# Perform the join using the country_3_letter_code
merged_df = resultats.join(gdp_spark_df, resultats['country_name'] == gdp_spark_df['country'], how='inner')

# Show the merged DataFrame
merged_df.show()

# Optionally, you can select specific columns if needed
selected_columns = merged_df.select(
    'country_name', 'discipline_title', 'event_title', 'medal_type', 
    'athlete_full_name', 'GDP_Nominal', 'GDP_PPP', 'GDP_per_capita_Nominal', 
    'GDP_per_capita_PPP', 'GDP_Growth_2020'
)

# Show the selected columns
selected_columns.show()

root
 |-- country: string (nullable = true)
 |-- GDP_Nominal: float (nullable = true)
 |-- GDP_PPP: float (nullable = true)
 |-- GDP_per_capita_Nominal: float (nullable = true)
 |-- GDP_per_capita_PPP: float (nullable = true)
 |-- GDP_Growth_2020: float (nullable = true)

root
 |-- Unnamed: 0: integer (nullable = true)
 |-- discipline_title: string (nullable = true)
 |-- event_title: string (nullable = true)
 |-- slug_game: string (nullable = true)
 |-- participant_type: string (nullable = true)
 |-- medal_type: string (nullable = true)
 |-- rank_equal: boolean (nullable = true)
 |-- rank_position: integer (nullable = true)
 |-- country_name: string (nullable = true)
 |-- country_3_letter_code: string (nullable = true)
 |-- athlete_url: string (nullable = true)
 |-- athlete_full_name: string (nullable = true)



24/05/30 14:38:55 WARN TaskSetManager: Stage 3883 contains a task of very large size (4943 KiB). The maximum recommended task size is 1000 KiB.


+----------+--------------------+--------------------+------------+----------------+----------+----------+-------------+------------+---------------------+--------------------+--------------------+------------+-----------+-------+----------------------+------------------+---------------+
|Unnamed: 0|    discipline_title|         event_title|   slug_game|participant_type|medal_type|rank_equal|rank_position|country_name|country_3_letter_code|         athlete_url|   athlete_full_name|     country|GDP_Nominal|GDP_PPP|GDP_per_capita_Nominal|GDP_per_capita_PPP|GDP_Growth_2020|
+----------+--------------------+--------------------+------------+----------------+----------+----------+-------------+------------+---------------------+--------------------+--------------------+------------+-----------+-------+----------------------+------------------+---------------+
|      1113|      Figure skating|           Ice Dance|beijing-2022|        GameTeam|         0|     false|           18|     ARMENIA|

24/05/30 14:38:57 WARN TaskSetManager: Stage 3889 contains a task of very large size (4943 KiB). The maximum recommended task size is 1000 KiB.


+------------+--------------------+--------------------+----------+--------------------+-----------+-------+----------------------+------------------+---------------+
|country_name|    discipline_title|         event_title|medal_type|   athlete_full_name|GDP_Nominal|GDP_PPP|GDP_per_capita_Nominal|GDP_per_capita_PPP|GDP_Growth_2020|
+------------+--------------------+--------------------+----------+--------------------+-----------+-------+----------------------+------------------+---------------+
|     ARMENIA|      Figure skating|           Ice Dance|         0|                 NaN|     12.339|  39.38|                  NULL|              NULL|         -7.571|
|     ARMENIA|       Alpine Skiing|  Men's Giant Slalom|         0|Harutyun HARUTYUNYAN|     12.339|  39.38|                  NULL|              NULL|         -7.571|
|     ARMENIA|Cross Country Skiing| Women's Sprint Free|         0|      Katya GALSTYAN|     12.339|  39.38|                  NULL|              NULL|         -7.571

In [298]:
# Convert the country names to uppercase and trim whitespace for consistency
gdp_spark_df = gdp_spark_df.withColumn('country', trim(upper(col('country'))))
resultats = resultats.withColumn('country_name', trim(upper(col('country_name'))))

# Filter out rows where medal_type is 0 
medal_resultats = resultats.filter(resultats.medal_type != '0')

# Aggregate the number of medals won by each country
medal_counts = medal_resultats.groupBy('country_name').agg(
    count('medal_type').alias('total_medals')
)

# Join the aggregated medal data with the GDP data
merged_df = medal_counts.join(gdp_spark_df, medal_counts['country_name'] == gdp_spark_df['country'], how='inner')

# Select relevant columns
final_df = merged_df.select(
    'country_name', 'total_medals', 'GDP_Nominal', 'GDP_PPP'
)

# Show the final DataFrame
print("Final DataFrame with Medal Counts and GDP Data:")
final_df.show()

# Calculate correlation between total_medals and GDP factors
correlations = {
    "GDP_Nominal": final_df.corr("total_medals", "GDP_Nominal"),
    "GDP_PPP": final_df.corr("total_medals", "GDP_PPP")
}

# Print correlation results
for factor, correlation in correlations.items():
    print(f"Correlation between total_medals and {factor}: {correlation}")


Final DataFrame with Medal Counts and GDP Data:


24/05/30 14:39:54 WARN TaskSetManager: Stage 3895 contains a task of very large size (4943 KiB). The maximum recommended task size is 1000 KiB.


+--------------+------------+-----------+-------+
|  country_name|total_medals|GDP_Nominal|GDP_PPP|
+--------------+------------+-----------+-------+
|       ARMENIA|          18|     12.339|  39.38|
|  SOUTH AFRICA|          89|    302.114|717.396|
|         JAPAN|         572|       NULL|   NULL|
|        UGANDA|          11|     37.613|106.106|
|        JORDAN|           3|     43.481|105.206|
|       BELARUS|         105|     60.201|189.922|
|     MAURITIUS|           1|     11.396| 25.712|
|        KUWAIT|           3|    107.936|203.259|
|     LITHUANIA|          26|     55.688|108.555|
|    AZERBAIJAN|          50|     42.607| 145.77|
|   PHILIPPINES|          14|    362.243|919.368|
|CZECH REPUBLIC|         100|    241.455|434.364|
|       ESTONIA|          44|     31.005| 50.162|
| CÔTE D'IVOIRE|           1|     61.402|144.631|
|          FIJI|           3|      4.316| 10.406|
|        CANADA|         548|       NULL|   NULL|
|        POLAND|         321|     594.18|   NULL|


24/05/30 14:40:11 WARN TaskSetManager: Stage 3901 contains a task of very large size (4943 KiB). The maximum recommended task size is 1000 KiB.
24/05/30 14:40:38 WARN TaskSetManager: Stage 3910 contains a task of very large size (4943 KiB). The maximum recommended task size is 1000 KiB.


Correlation between total_medals and GDP_Nominal: 0.1942935741877944
Correlation between total_medals and GDP_PPP: 0.046927339342527186


### Réponse Q BONUS 
Il existe une faible corrélation positive entre le nombre de médailles et le PIB nominal (
0,194
0,194), ce qui signifie que les pays ayant un PIB nominal plus élevé ont tendance à remporter plus de médailles, mais la relation n’est pas très forte.
La corrélation entre le nombre de médailles et le PIB (PPA) est encore plus faible (
0,047
0,047), suggérant qu’il n’y a pratiquement aucune relation.